# Spatial dataset visualization

This notebook contains code for visualizing the highest expression gene in each of the eleven spatial transcriptomics datasets along the two-dimensional spatial coordinates.

The notebook requires having the dataset folders formatted and in "DataUpload" top-level directory in the current working directory. The data can be retrieved from the Zenodo repository in our manuscript (DOI: 10.5281/zenodo.8259942).

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib
matplotlib.rcParams['pdf.fonttype'] = 42
matplotlib.rcParams['ps.fonttype'] = 42
import scanpy as sc
import seaborn as sns
import anndata as ad
import pickle
import os

from TISSUE.tissue.main import *

In [2]:
dataset_mapper = {"Dataset3": "Mouse Hipp. (seqFISH)",
                  "Dataset9": "Mouse VISP (MERFISH)",
                  "Dataset11": "Mouse PC (STARmap)", 
                  "Dataset12": "Human MTG (ISS)",
                  "Dataset13": "Mouse VISP (ISS)",
                  "Dataset14": "Drosophila Embryo (FISH)",
                  "Dataset15": "Mouse SC (osmFISH)",
                  "Dataset17": "Mouse VISP (ExSeq)",
                  "Axolotl": "Axolotl Brain (Stereo-seq)",
                  "Dataset7": "Human U2OS (MERFISH)",
                  "Dataset1": "Mouse Gast. (seqFISH)"}

In [ ]:
dataset_names = ["Dataset3", "Dataset9", "Dataset11", "Dataset12", "Dataset13", "Dataset14", "Dataset15", "Dataset17", "Dataset1", "Dataset7", "Axolotl"]


# make plot for each dataset
for dataset_name in dataset_names:
    
    # load in spatial data
    adata = load_spatial_data("DataUpload/"+dataset_name+"/Spatial_count.txt",
                             "DataUpload/"+dataset_name+"/Locations.txt")
    
    # log counts
    adata.X = np.log1p(adata.X)
    
    # special handling of 3D Dataset14 (2D projection)
    if dataset_name == "Dataset14":
        adata.obsm["spatial"] = adata.obsm["spatial"][:,np.array([0,2])]
    
    # get max expression gene
    max_gene = adata.var_names[np.where(np.sum(adata.X, axis=0) == np.sum(adata.X, axis=0).max())[0][0]]
    
    # make plot
    sc.set_figure_params(vector_friendly=True, dpi=500, figsize=(7,5), fontsize=16)
    sc.pl.embedding(adata, "spatial", color=max_gene, cmap="Reds", show=False)
    plt.title(dataset_mapper[dataset_name]+"\n"+max_gene.capitalize(), fontsize=20)
    plt.axis('off')
    plt.savefig(f"plots/spatial_max_gene_{dataset_name}_{max_gene}.pdf", bbox_inches="tight", dpi=300)